In [2]:
ls /home/ubuntu/mne-hcp-data/*csv

/home/ubuntu/mne-hcp-data/hcp_restricted_data.csv
/home/ubuntu/mne-hcp-data/hcp_unrestricted_data.csv
/home/ubuntu/mne-hcp-data/motor_sensor_unrelated_subjects.csv
/home/ubuntu/mne-hcp-data/motor_source_unrelated_subjects.csv
/home/ubuntu/mne-hcp-data/rs_sensor_unrelated_subjects.csv
/home/ubuntu/mne-hcp-data/rs_source_unrelated_subjects.csv
/home/ubuntu/mne-hcp-data/storym_sensor_unrelated_subjects.csv
/home/ubuntu/mne-hcp-data/storym_source_unrelated_subjects.csv
/home/ubuntu/mne-hcp-data/unrel_subject_processing_map.csv
/home/ubuntu/mne-hcp-data/wm_sensor_unrelated_subjects.csv
/home/ubuntu/mne-hcp-data/wm_source_unrelated_subjects.csv


In [3]:
df = pd.read_csv(op.expanduser('~/mne-hcp-data/unrel_subject_processing_map.csv'))

In [4]:
subjects_rs = df.Subject[df.unrelated_rs_source].tolist()

In [5]:
aws_key, aws_secret = get_aws_credentials(
    op.join(op.expanduser("~"), "mne-hcp-aws", 'aws_hcp_details.csv'))

In [15]:
get_s3_keys_anatomy?

In [16]:
hcp_prefix = 's3://hcp-openaccess/HCP_900'
s3_keys = list()
max_runs = 3
run_inds = [0, 1, 2]
for subject in subjects_rs:
    hcp_data_types = [
        'rest',
#         'noise_empty_room'
    ]

    hcp_outputs = [
#         'bads',
#         'ica',
        'raw',
    ]

    # allow for downloading fewer data
    s3_keys = get_s3_keys_meg(
        subject,
        data_types=hcp_data_types,
        hcp_path_bucket='HCP_900',
        outputs=hcp_outputs,
        run_inds=run_inds[:max_runs])
#     s3_keys = [kk for kk in s3_keys if 'config' in kk]

    s3_keys += get_s3_keys_anatomy(
        subject,
        hcp_path_bucket='HCP_900')
    download_from_s3_bucket(
        bucket='hcp-openaccess',
        out_path='/mnt/HCP', key_list=s3_keys, aws_key=aws_key,
        aws_secret=aws_secret,
        prefix='HCP_900')
    break

Elapsed time downloading hcp-openaccess from s3 00:01:38


In [18]:
ls /mnt/HCP/105923/

MEG/  T1w/  unprocessed/


In [6]:
aws_key_dyn, aws_secret_dyn = get_aws_credentials(
    op.join(op.expanduser("~"), "mne-hcp-aws", 'aws_details.csv'))

In [7]:
hcp_prefix = 's3://hcp-openaccess/HCP_900'
s3_keys = list()
max_runs = 3
run_inds = [0, 1, 2]
for subject in subjects_rs:
    s3_keys = s3_glob(key_pattern='hcp-meg/%s/rest*raw.fif' % subject, bucket='hcp-meg-data',
         aws_key=aws_key_dyn, aws_secret=aws_secret_dyn, prefix='')
    download_from_s3_bucket(
        bucket='hcp-meg-data',
        out_path='/mnt', key_list=s3_keys,
        aws_key=aws_key_dyn,
        aws_secret=aws_secret_dyn,
        prefix='')
    break

Elapsed time downloading hcp-meg-data from s3 00:00:32
